# 1. Passo a passo

1. Baixar a base de dados.
2. Criar amostras com reposição e armazená-las em uma lista.
3. Para cada amostra:
  *   Dividir os dados em conjuntos de treino e teste, selecionando aleatoriamente apenas duas colunas.
  *   Treinar o modelo usando o conjunto de treino.
  *   Criar previsões usando o conjunto de teste, garantindo que apenas as colunas utilizadas para treinar o modelo sejam usadas para testar.
4. Consolidar todas as previsões em um único dataframe.
5. Realizar a agregação das previsões com base nas suas frequências.

# 2. Descrição

O Random Forest é um método de ensemble learning utilizado para melhorar a eficácia dos resultados de modelos de aprendizado de máquina. Ele consiste em criar múltiplas amostras com repetição a partir do conjunto de dados original. Cada amostra deve conter um número reduzido de colunas, geralmente igual à raiz quadrada do total de colunas, e essas colunas devem ser escolhidas aleatoriamente.

Após a criação das amostras, cada uma é usada para treinar um modelo individual. Cada modelo, então, gera previsões. Utilizamos uma técnica de agregação para determinar o melhor resultado a partir dessas previsões. No caso de variáveis discretas, o melhor resultado é aquele que aparece com maior frequência entre os diferentes modelos. Para variáveis contínuas, o melhor resultado é a média de todos os resultados.

# 3. Qual a diferença entre Bagging e Random Forest?

Bagging pode ser aplicado a qualquer modelo e envolve a criação de vários modelos treinados em subconjuntos aleatórios dos dados originais, combinando seus resultados para maior estabilidade.Ja o Random Forest é uma aplicação que utiliza apenas árvores de decisão, adicionando aleatoriedade ao selecionar subconjuntos de características para cada árvore, o que aumenta a diversidade e reduz a correlação entre as árvores.

# 4. Implementar em python o Random Forest

In [ ]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = sns.load_dataset("penguins")
df.dropna(inplace=True)
df['species'] = df['species'].map({'Adelie': 0, 'Chinstrap':1, 'Gentoo': 2 })
df['sex'] = df['sex'].map({'Male':0, 'Female':1})
df


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,Torgersen,39.1,18.7,181.0,3750.0,0
1,0,Torgersen,39.5,17.4,186.0,3800.0,1
2,0,Torgersen,40.3,18.0,195.0,3250.0,1
4,0,Torgersen,36.7,19.3,193.0,3450.0,1
5,0,Torgersen,39.3,20.6,190.0,3650.0,0
...,...,...,...,...,...,...,...
338,2,Biscoe,47.2,13.7,214.0,4925.0,1
340,2,Biscoe,46.8,14.3,215.0,4850.0,1
341,2,Biscoe,50.4,15.7,222.0,5750.0,0
342,2,Biscoe,45.2,14.8,212.0,5200.0,1


In [ ]:
n_col = math.floor(np.sqrt(len(df.columns)))

In [ ]:
amostras = []

for i in range(20):
    amostra = df.sample(len(df), replace=True, random_state=i)
    amostras.append(amostra)

amostras

[     species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
 178        1      Dream            50.5           18.4              200.0   
 53         0     Biscoe            42.0           19.5              200.0   
 123        0  Torgersen            41.4           18.5              202.0   
 198        1      Dream            50.1           17.9              190.0   
 332        2     Biscoe            43.5           15.2              213.0   
 ..       ...        ...             ...            ...                ...   
 233        2     Biscoe            48.4           14.6              213.0   
 281        2     Biscoe            46.2           14.9              221.0   
 298        2     Biscoe            45.2           13.8              215.0   
 57         0     Biscoe            40.6           18.8              193.0   
 164        1      Dream            47.0           17.3              185.0   
 
      body_mass_g  sex  
 178       3400.0    1  
 53        4

In [ ]:

amostras[0].drop(['species', 'island'], axis=1).sample(2, axis='columns')

,sex,bill_length_mm
178,1,50.5
53,0,42.0
123,0,41.4
198,1,50.1
332,1,43.5
...,...,...
233,0,48.4
281,0,46.2
298,1,45.2
57,0,40.6


In [ ]:
splits = []
for amostra in amostras:
    #print(amostra['island'])
    X = amostra.drop(['species', 'island'], axis=1).sample(2, axis='columns')
    y = amostra['species']
    splits.append(train_test_split(X, y, test_size=0.3))

splits

[[     bill_depth_mm  body_mass_g
  258           13.6       4350.0
  167           19.6       4050.0
  18            18.4       3325.0
  115           18.3       4075.0
  46            19.0       3425.0
  ..             ...          ...
  250           15.3       5250.0
  154           19.2       3650.0
  134           17.6       3425.0
  333           16.3       5500.0
  157           17.8       3950.0
  
  [233 rows x 2 columns],
       bill_depth_mm  body_mass_g
  221           16.3       5700.0
  86            19.5       3800.0
  263           15.0       4750.0
  340           14.3       4850.0
  33            18.9       3900.0
  ..             ...          ...
  36            20.0       3950.0
  94            17.3       3300.0
  188           18.3       3850.0
  303           15.9       5350.0
  330           15.2       5000.0
  
  [100 rows x 2 columns],
  258    2
  167    1
  18     0
  115    0
  46     0
        ..
  250    2
  154    1
  134    0
  333    2
  157    1
  Nam

In [ ]:
models=[]

for split in splits:
    model = DecisionTreeClassifier(max_depth=5, min_samples_leaf=20)
    model.fit(split[0], split[2])
    models.append(model)

models[0].feature_names_in_

array(['bill_depth_mm', 'body_mass_g'], dtype=object)

In [ ]:
test_X= df.drop(['species', 'island'], axis=1).sample(1, random_state=241)
test_X

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
157,45.2,17.8,198.0,3950.0,1


In [ ]:
predictions = []

for j in range(len(models)):
    features = list(models[j].feature_names_in_)
    new_X = test_X.loc[:,features]
    y_pred = models[j].predict(new_X)
    predictions.append(y_pred[0])

# for model in models:
#     for split in splits:
#         print(model.
#         y_pred = model.predict(split[1])
#     predictions.append(y_pred[0])
predictions

[1, 1, 1, 0, 0, 2, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1]

In [ ]:
final_prediction = pd.DataFrame(predictions).value_counts().idxmax()[0]
final_prediction

1